## Exploratory Data Analysis

In [1]:
import pandas as pd

In [2]:
beers = pd.read_csv('beers.csv')

In [3]:
beers.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [4]:
beers.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
beers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 7 columns):
abv           2348 non-null float64
ibu           1405 non-null float64
id            2410 non-null int64
name          2410 non-null object
style         2405 non-null object
brewery_id    2410 non-null int64
ounces        2410 non-null float64
dtypes: float64(3), int64(2), object(2)
memory usage: 131.9+ KB


In [6]:
beers['brewery_id'].nunique()

558

In [7]:
breweries = pd.read_csv('breweries.csv')

In [8]:
breweries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 4 columns):
Unnamed: 0    558 non-null int64
name          558 non-null object
city          558 non-null object
state         558 non-null object
dtypes: int64(1), object(3)
memory usage: 17.6+ KB


In [9]:
breweries.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [10]:
breweries = breweries.rename(columns={"Unnamed: 0": "brewery_id"})
breweries.head()

,brewery_id,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [11]:
breweries['brewery_id'].nunique()

558

In [12]:

# breweries.join(beers, on="brewery_id", how="left")
# breweries.join(beers.set_index(['brewery_id'], verify_integrity=True),
#                on=['brewery_id'], how='left')